In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score

In [8]:
df = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv', encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [35]:
df = df[['v1','v2']]
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [37]:
# Replace special characters with nothing and apply lowercase in string fir each row
special_char = '@_!#$%^&*()<>?/\|}{~:.,;[]'
for i, row in df.iterrows():
    df.iloc[i, 1] = ''.join(x for x in df.iloc[i, 1] if not x in special_char)
    df.iloc[i, 1] = df.iloc[i, 1].lower()
df.head()

,v1,v2
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i don't think he goes to usf he lives arou...


In [48]:
# Get uniques words from all text in sms dataframe
all_words = []
for i, row in df.iterrows():
    list_words = df.iloc[i, 1].split(' ')
    all_words = all_words + list_words
all_words = pd.DataFrame(all_words)
print('n all words:', all_words.shape[0])
uniques_words = all_words[0].unique()
print('n uniques words:', uniques_words.shape[0])

n all words: 86961
n uniques words: 9877


In [126]:
# Create X df with the count for each word from the sms text
y = df.iloc[:,0]
X = pd.DataFrame(0, index=df.index, columns=uniques_words)
for i, row in df.iterrows() :
    list_words = row[1].split(' ')
    for word in list_words :
        X.loc[i,[word]] = X.loc[i,[word]] + [1 for col in X.columns if word == col]
X

,go,until,jurong,point,crazy,available,only,in,bugis,n,...,heap,lowes,salesman,å£750,087187272008,now1,pity,soany,suggestions,bitching
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,1,1,0,0,0,0
5568,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5569,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,1,1,0
5570,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [112]:
X.loc[0,['go']]

go    1
Name: 0, dtype: int64

In [32]:
import numpy as np
rng = np.random.RandomState(1)
X = rng.randint(5, size=(6, 100))
y = np.array([1, 2, 3, 4, 5, 6])
X

array([[3, 4, 0, 1, 3, 0, 0, 1, 4, 4, 1, 2, 4, 2, 4, 3, 4, 2, 4, 2, 4, 1,
        1, 0, 1, 1, 1, 1, 0, 4, 1, 0, 0, 3, 2, 1, 0, 3, 1, 1, 3, 4, 0, 1,
        3, 4, 2, 4, 0, 3, 1, 2, 0, 4, 1, 2, 2, 1, 0, 1, 3, 4, 3, 1, 3, 0,
        0, 2, 2, 1, 3, 4, 2, 0, 0, 1, 1, 3, 0, 0, 4, 2, 4, 3, 3, 0, 3, 4,
        3, 4, 4, 4, 1, 0, 4, 2, 0, 2, 4, 1],
       [1, 0, 2, 4, 4, 0, 4, 1, 4, 1, 0, 2, 3, 1, 2, 4, 4, 2, 2, 0, 1, 2,
        2, 0, 1, 2, 4, 0, 1, 2, 1, 4, 2, 0, 0, 1, 0, 1, 3, 1, 1, 4, 4, 3,
        0, 3, 0, 3, 1, 2, 4, 0, 0, 3, 1, 1, 0, 0, 4, 2, 3, 4, 2, 0, 3, 3,
        1, 2, 4, 3, 0, 0, 4, 2, 4, 2, 0, 3, 0, 0, 4, 2, 1, 0, 4, 3, 0, 1,
        2, 4, 4, 3, 3, 3, 3, 2, 3, 3, 4, 3],
       [2, 4, 4, 0, 3, 3, 0, 3, 1, 0, 2, 2, 2, 0, 2, 1, 4, 0, 4, 4, 1, 3,
        1, 4, 1, 2, 1, 0, 0, 2, 4, 1, 0, 0, 3, 1, 0, 4, 3, 2, 3, 4, 4, 3,
        0, 0, 0, 4, 1, 4, 1, 2, 2, 4, 3, 4, 4, 0, 3, 2, 4, 3, 4, 2, 3, 0,
        2, 1, 3, 2, 0, 1, 4, 1, 3, 3, 1, 2, 0, 2, 4, 0, 2, 4, 3, 4, 3, 0,
        4, 2, 2, 4, 1,